In [1]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python numpy pandas argparse emoji tqdm matplotlib

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [1]:
import os

# Create required directories
os.makedirs("weights", exist_ok=True)
os.makedirs("config", exist_ok=True)
os.makedirs("data", exist_ok=True)
os.makedirs("vehicles-on-lanes", exist_ok=True)

# Download YOLOv3 weights
!wget https://pjreddie.com/media/files/yolov3.weights -O weights/yolov3.weights

# Download YOLOv3 config
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg -O config/yolov3.cfg

# Create class names file (customize if needed)
!echo -e "car\ntruck\nmotorbike\nbicycle\nautorickshaw" > data/idd.names

--2025-04-01 16:36:08--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘weights/yolov3.weights’

weights/yolov3.weig 100%[===================>] 236.52M  5.10MB/s    in 41s     

2025-04-01 16:36:50 (5.77 MB/s) - ‘weights/yolov3.weights’ saved [248007048/248007048]

--2025-04-01 16:36:50--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘config/yolov3.cfg’

config/yolov3.cfg   100%[===================>]   8.15K  --.

In [6]:
import os
import cv2
import torch
import argparse
from torch.autograd import Variable

# ====================== SETUP FOLDERS & FILES ======================
os.makedirs("weights", exist_ok=True)
os.makedirs("config", exist_ok=True)
os.makedirs("data", exist_ok=True)
os.makedirs("vehicles-on-lanes", exist_ok=True)

# Download YOLOv3 files
!wget https://pjreddie.com/media/files/yolov3.weights -O weights/yolov3.weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg -O config/yolov3.cfg
!echo -e "car\ntruck\nmotorbike\nbicycle\nautorickshaw" > data/idd.names

# ====================== CUSTOM UTIL FUNCTIONS ======================
def load_classes(namesfile):
    with open(namesfile, 'r') as f:
        names = f.read().split('\n')
    return list(filter(None, names))

def preparing_image(img, inp_dim):
    img = cv2.resize(img, (inp_dim, inp_dim))
    img = img[:, :, ::-1].transpose((2, 0, 1))  # BGR to RGB
    img = torch.from_numpy(img).float().div(255.0).unsqueeze(0)
    return img

def non_max_suppression(prediction, confidence, num_classes, nms_conf=0.3):
    # Simplified NMS (replace with actual implementation if needed)
    return prediction

def avg_signal_oc_time(counts):
    return max(counts) * 2  # Example: 2 seconds per vehicle

def switch_signal(lane, time):
    print(f"\033[92mSwitching signal to Lane {lane} for {time} seconds.\033[0m")

# ====================== MAIN YOLO DETECTION ======================
def run_detection(args):
    classes = load_classes("data/idd.names")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Simplified Darknet model (replace with actual YOLOv3 if needed)
    model = torch.nn.Sequential(
        torch.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU()
    ).to(device)

    # Process images
    for img_path in os.listdir(args.images):
        img = cv2.imread(os.path.join(args.images, img_path))
        img_tensor = preparing_image(img, int(args.reso)).to(device)

        with torch.no_grad():
            prediction = model(img_tensor)

        # Fake detection results (replace with actual YOLO output)
        vehicle_count = torch.randint(1, 10, (1,)).item()
        print(f"Lane {img_path} - Vehicles detected: {vehicle_count}")

    # Simulate signal switching
    switch_signal(1, avg_signal_oc_time([5, 10]))  # Example counts

# ====================== RUN ======================
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--images", default="vehicles-on-lanes")
    parser.add_argument("--reso", default="416")
    args = parser.parse_args([])  # Use empty list for Colab

    print('\033[1m\033[91mKickstarting YOLO...\n')
    run_detection(args)

--2025-04-01 16:44:28--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘weights/yolov3.weights’

weights/yolov3.weig 100%[===================>] 236.52M  8.01MB/s    in 36s     

2025-04-01 16:45:06 (6.48 MB/s) - ‘weights/yolov3.weights’ saved [248007048/248007048]

--2025-04-01 16:45:06--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘config/yolov3.cfg’

config/yolov3.cfg   100%[===================>]   8.15K  --.

In [8]:
!python traffic_detection.py --images vehicles-on-lanes --weights weights/yolov3.weights --cfg config/yolov3.cfg

Kickstarting YOLO...

Traceback (most recent call last):
  File "/content/traffic_detection.py", line 19, in <module>
    from util.parser import load_classes
ModuleNotFoundError: No module named 'util'
